# Random Sampling in a Hyperplane Arrangement Complement

### Import Packages

In [43]:
using LinearAlgebra
include("lambda_cyclic.jl")

getRank (generic function with 1 method)

### Generate a random point

In [44]:
p(n) = [rand(Float64)-.5 for j=1:n]

p (generic function with 1 method)

### Generate the 3-Sunlet hyperplanes

In [45]:
function getH(k)
    H = []
    v = zero(rand(2*k))
    v[1] = 1
    push!(H, v)
    
    for i=2:k
        for j=(k+1):(2*k)
            v = zero(rand(2*k))
            v[i] = v[j] = 1
            push!(H, v)
            
            w = zero(rand(2*k))
            w[i] = 1
            w[j] = -1
            push!(H, w)
        end
    end
    return H
end

getH (generic function with 1 method)

In [46]:
H3 = [[1 0 0 0 0 0], [0 1 0 -1 0 0], [0 1 0 0 -1 0], [0 1 0 0 0 -1], [0 0 1 1 0 0], [0 0 1 0 1 0], [0 0 1 0 0 1]];
H3

7-element Vector{Matrix{Int64}}:
 [1 0 … 0 0]
 [0 1 … 0 0]
 [0 1 … -1 0]
 [0 1 … 0 -1]
 [0 0 … 0 0]
 [0 0 … 1 0]
 [0 0 … 0 1]

In [47]:
coords_3 = ["mu_0", "mu_1", "mu_2", "eta_{10}", "eta_{21}", "eta_{02}"]
coords_3[[i for i in eachindex(H3[1]) if H3[1][i] != 0]]

1-element Vector{String}:
 "mu_0"

In [48]:
for h in eachindex(H3)
    H = H3[h]
    print("\$H_", h, "\$ & ")
    for j in eachindex(H)
        print(H[j], " & ")
    end
    vars = [i for i in eachindex(H) if H[i] != 0]
    coord = coords[vars]
    coeff = H[vars]
    print("\$")
    for v in eachindex(vars)
        if v != length(vars)
            if coeff[v+1] > 0
                print("\\", coord[v], " + ")
            else
                print("\\", coord[v], " - ")
            end
        else
            print("\\", coord[v], " = 0\$ \\")
        end
    end
    println()
end

$H_1$ & 1 & 0 & 0 & 0 & 0 & 0 & 

LoadError: UndefVarError: coords not defined

In [49]:
coords_4 = ["mu_0", "mu_1", "mu_2", "mu_3", "eta_{10}", "eta_{21}", "eta_{32}", "eta_{03}"]

8-element Vector{String}:
 "mu_0"
 "mu_1"
 "mu_2"
 "mu_3"
 "eta_{10}"
 "eta_{21}"
 "eta_{32}"
 "eta_{03}"

In [50]:
for h in eachindex(H3)
    H = H3[h]
    print("\$H_", h, "\$ & ")
    for j in eachindex(H)
        print(H[j], " & ")
    end
    vars = [i for i in eachindex(H) if H[i] != 0]
    coord = coords[vars]
    coeff = H[vars]
    print("\$")
    for v in eachindex(vars)
        if v != length(vars)
            if coeff[v+1] > 0
                print("\\", coord[v], " + ")
            else
                print("\\", coord[v], " - ")
            end
        else
            print("\\", coord[v], " = 0\$ \\")
        end
    end
    println()
end

$H_1$ & 1 & 0 & 0 & 0 & 0 & 0 & 

LoadError: UndefVarError: coords not defined

In [51]:
function HToLatex(Hs, coords)
    for h in eachindex(Hs)
        H = Hs[h]
        print("\$H_", h, "\$ & ")
        for j in eachindex(H)
            print(H[j], " & ")
        end
        
        vars = [i for i in eachindex(H) if H[i] != 0]
        coord = coords[vars]
        coeff = H[vars]
        print("\$")
        for v in eachindex(vars)
            if v != length(vars)
                if coeff[v+1] > 0
                     print("\\", coord[v], " + ")
                else
                    print("\\", coord[v], " - ")
                end
            else
                print("\\", coord[v], " = 0\$ \\")
            end
        end
        println()
    end
end

HToLatex (generic function with 1 method)

In [52]:
HToLatex(H4, coords_4)

LoadError: UndefVarError: H4 not defined

### Inequality vector

In [53]:
ineq(v, H) = [LinearAlgebra.dot(v, h) > 0 for h in H]

ineq (generic function with 1 method)

In [54]:
neighbors(I, J) = (sum(I .⊻ J) == 1)

neighbors (generic function with 1 method)

### Sampling Algorithm

In [55]:
function sample(k::Int, H, N::Int)
    sample = Dict()
    for i=1:N
        v = p(2*k)
        v[end] = -sum(v[(k+1):(end-1)])
        sample[ineq(v, H)] = v
    end
    return sample
end

sample (generic function with 1 method)

## Example $n = 3$

In [56]:
n = 3
@time s3 = sample(3, H3, 2^20); ## get the sample points (< 2 seconds usually)
println(length(s3))
ranks = [getRank(s, n) for s in collect(values(s3))]; ## get the rank for each sample point
R = Set(ranks)
println("R=", R)
counts = [sum([r==k for r in ranks]) for k in R]; ## tabulate the ranks
priors = [c/sum(counts) for c in counts]; ## compute priors (% of samples with given rank)
counts

  0.538691 seconds (3.30 M allocations: 263.452 MiB, 8.11% gc time, 20.45% compilation time)
92
R=Set([7, 9, 8])


3-element Vector{Int64}:
  4
 64
 24

### Test Neighbors

In [57]:
for I in collect(keys(s3))
    s = s3[I]
    curr_rank = getRank(s, n)
    for J in collect(keys(s3))
        if neighbors(I, J)
            neighbor = s3[J]
            n_rank = getRank(neighbor, n)
            print(abs(curr_rank - n_rank))
        end
    end
end

10010001011111011110111100000101101111000000000000001101111110110000000000000011110110100000111101111101111010010100110111101001000100011001101001011000100010000000110001111011111100000000011001111101011000110011000011001111101000000000000000010101111110010100101001010111100000000111110101000000001111100010101111110100010011011110010111011100000001001001111100000000111110010110100101000111110010100000000000000110111100011110000001100011111101100000000001110111111101111000000000011000110001010000011001101100010010111011

### Parochial Chambers

In [60]:
p1 = s3[[1, 1, 1, 1, 1, 1, 1]];
p2 = s3[[0, 0, 0, 0, 0, 0, 0]];
r1 = getRank(p1, n);
r2 = getRank(p2, n);
println(r1, r2)
parochial(s3, 7)
# keys(s3)

77
7 7

In [59]:
function parochial(S, k)
    p0 = S[zeros(Bool, k)]
    p1 = S[ones(Bool, k)]
    r0 = getRank(p0, n)
    r1 = getRank(p1, n)
    print(r0, ' ', r1)
end

parochial (generic function with 1 method)

In [61]:
p_avg = (p1 .+ p2)./2; ## average the sample representing parochial chambers
r_avg = getRank(p_avg, n) ## the average happens to have full rank

9

### Low Rank Chambers

In [62]:
low_rank = []
s3_keys = collect(keys(s3))
for s in eachindex(s3_keys)
    if ranks[s] == 7
        push!(low_rank, s)
    end
end
length(low_rank)

4

In [63]:
## two of the chambers are parochial
s3_keys[low_rank]

4-element Vector{Any}:
 Bool[1, 1, 1, 1, 1, 1, 1]
 Bool[1, 0, 0, 0, 0, 0, 0]
 Bool[0, 1, 1, 1, 1, 1, 1]
 Bool[0, 0, 0, 0, 0, 0, 0]

### Testing to_lambda function

In [64]:
s0 = collect(values(s3))[1];
ineq(s0, H3)

7-element Vector{Bool}:
 0
 0
 0
 1
 1
 1
 1

In [65]:
G = [[0, 2, 1], [1, 1, 1], [1, 0, 2], [1, 2, 0], [2, 2, 2], [2, 1, 0], [2, 0, 1]]
for g in G
    print(computeLambda(g, m1, to_lambda(s0[1:3], s0[4:6], 3), 3), ' ')
    println(computeLambda(g, m2, to_lambda(s0[1:3], s0[4:6], 3), 3))
end

LoadError: UndefVarError: m1 not defined

In [66]:
s0

6-element Vector{Float64}:
 -0.13073104832537963
 -0.16128647602045898
  0.3365483048943869
  0.017528504652706944
  0.14811789844717083
 -0.16564640309987777

## Example $n = 4$

### Generate the Hyperplanes

In [67]:
## mu 0 1 2 3 eta 10 21 32 03
H4 = [[1 0 0 0 0 0 0 0], [0 0 1 0 0 -1 -1 0], [0 0 1 0 -1 -1 0 0], [0 0 1 0 0 1 1 0], [0 0 1 0 1 1 0 0], [0 1 0 0 0 0 -1 0], [0 1 0 0 0 -1 0 0], [0 1 0 0 -1 0 0 0], [0 1 0 0 0 0 0 -1], [0 0 0 1 0 0 0 1], [0 0 0 1 0 0 1 0], [0 0 0 1 0 1 0 0], [0 0 0 1 1 0 0 0]]

13-element Vector{Matrix{Int64}}:
 [1 0 … 0 0]
 [0 0 … -1 0]
 [0 0 … 0 0]
 [0 0 … 1 0]
 [0 0 … 0 0]
 [0 1 … -1 0]
 [0 1 … 0 0]
 [0 1 … 0 0]
 [0 1 … 0 -1]
 [0 0 … 0 1]
 [0 0 … 1 0]
 [0 0 … 0 0]
 [0 0 … 0 0]

### Generate Sample Points

In [68]:
## 2328

In [69]:
function sample_info(n, H, N)
    @time S = sample(n, H, N)
    println(length(S))
    ranks = [getRank(s, n) for s in collect(values(S))];
    R = sort(collect(Set(ranks)))
    println("R=", R)
    counts = [sum([r==k for r in ranks]) for k in R]; ## tabulate the ranks
    println(counts)
    priors = [100*c/sum(counts) for c in counts]; ## compute priors (% of samples with given rank)
    println(priors)
    return S
end

sample_info (generic function with 1 method)

In [70]:
s4 = sample_info(4, H4, 2^20);

  0.670202 seconds (3.15 M allocations: 272.112 MiB, 10.65% gc time)
2328
R=[10, 11, 12, 13, 14, 15, 16]
[4, 48, 180, 496, 864, 624, 112]
[0.1718213058419244, 2.0618556701030926, 7.731958762886598, 21.305841924398624, 37.11340206185567, 26.804123711340207, 4.810996563573883]


### Check Neighbors

In [71]:
function check_neighbors(S, n)
    for I in collect(keys(S))
        s = S[I]
        curr_rank = getRank(s, n)
        for J in collect(keys(S))
            if neighbors(I, J)
                neighbor = S[J]
                n_rank = getRank(neighbor, n)
                diff = abs(curr_rank - n_rank)
                if diff > 1
                    print(diff)
                end
            end
        end
    end
end

check_neighbors (generic function with 1 method)

In [72]:
check_neighbors(s4, 4)

### Check Parochial Chambers

In [73]:
p0 = s4[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]];
p1 = s4[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]];
r0 = getRank(p0, 4)
r1 = getRank(p1, 4)
print(r0, ' ', r1)

10 10

### Low Rank Chambers

In [74]:
low_rank_4 = []
for s in collect(keys(s4))
    if getRank(s4[s], 4) == 10
        push!(low_rank_4, s)
    end
end
length(low_rank)

4

In [75]:
low_rank_4

4-element Vector{Any}:
 Bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 Bool[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 Bool[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

### Dictionary of ranks => inequalities

In [76]:
rdict_3 = Dict([r => [s3_keys[s] for s in eachindex(collect(keys(s3))) if ranks[s] == r] for r in R])

Dict{Int64, Vector{Vector{Bool}}} with 3 entries:
  7 => [[1, 1, 1, 1, 1, 1, 1], [1, 0, 0, 0, 0, 0, 0], [0, 1, 1, 1, 1, 1, 1], [0…
  9 => [[0, 0, 0, 1, 1, 1, 1], [1, 1, 0, 1, 0, 0, 0], [0, 1, 1, 0, 1, 0, 1], [1…
  8 => [[1, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0], [1, 1, 1, 1, 1, 0, 1], [1…

In [77]:
length(rdict_3[9])

64

In [78]:
function rankToIDict(S, R)
    rToI = Dict()
    K = collect(keys(S))
    for r in Set(R)
        rToI[r] = [K[s] for s in eachindex(K) if R[s] == r]
    end
    return rToI
end

rankToIDict (generic function with 1 method)

In [79]:
rdict_4 = rankToIDict(s4, [getRank(s, 4) for s in collect(values(s4))])

Dict{Any, Any} with 7 entries:
  13 => Vector{Bool}[[0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1…
  16 => Vector{Bool}[[1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0], [0, 0, 1, 1, 0, 0…
  15 => Vector{Bool}[[1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0], [0, 0, 1, 1, 0, 0…
  11 => Vector{Bool}[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0], [0, 0, 0, 0, 0, 0…
  10 => Vector{Bool}[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0…
  12 => Vector{Bool}[[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [1, 1, 1, 1, 1, 1…
  14 => Vector{Bool}[[1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0], [1, 1, 1, 0, 0, 0…

In [80]:
for r in collect(keys(rdict_4))
    println(length(rdict_4[r]))
end

496
112
624
48
4
180
864


In [81]:
function printRanks(RI)
    for r in sort(collect(keys(RI)))
        println("## rank = ", r)

        count = 0
        for I in RI[r]
            if count % 5 == 0
                println()
            end
            for b in I
                print(Int(b), ' ')
            end
            println()
            count += 1
        end

        println()
    end
end

printRanks (generic function with 1 method)

In [82]:
function latexRanks(RI)
    count = 1
    for r in sort(collect(keys(RI)))
        for I in RI[r]
            print(count, " & ")
            for i in eachindex(I)
                if i < length(I)
                    print(Int(I[i]), " & ")
                else
                    print(Int(I[i]), " & ", r, " & \\\\")
                end
            end
            println()
            count += 1
        end
    end
end

latexRanks (generic function with 1 method)

In [83]:
latexRanks(rdict_3)

1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 7 & \\
2 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 7 & \\
3 & 0 & 1 & 1 & 1 & 1 & 1 & 1 & 7 & \\
4 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 7 & \\
5 & 1 & 0 & 1 & 0 & 0 & 0 & 0 & 8 & \\
6 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 8 & \\
7 & 1 & 1 & 1 & 1 & 1 & 0 & 1 & 8 & \\
8 & 1 & 1 & 1 & 1 & 1 & 1 & 0 & 8 & \\
9 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 8 & \\
10 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 8 & \\
11 & 1 & 1 & 1 & 1 & 0 & 1 & 1 & 8 & \\
12 & 0 & 1 & 0 & 1 & 1 & 1 & 1 & 8 & \\
13 & 1 & 0 & 1 & 1 & 1 & 1 & 1 & 8 & \\
14 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 8 & \\
15 & 0 & 1 & 1 & 0 & 1 & 1 & 1 & 8 & \\
16 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 8 & \\
17 & 0 & 1 & 1 & 1 & 1 & 0 & 1 & 8 & \\
18 & 0 & 1 & 1 & 1 & 1 & 1 & 0 & 8 & \\
19 & 1 & 1 & 0 & 0 & 0 & 0 & 0 & 8 & \\
20 & 1 & 0 & 0 & 0 & 1 & 0 & 0 & 8 & \\
21 & 0 & 1 & 1 & 1 & 0 & 1 & 1 & 8 & \\
22 & 0 & 0 & 1 & 1 & 1 & 1 & 1 & 8 & \\
23 & 1 & 0 & 0 & 0 & 0 & 0 & 1 & 8 & \\
24 & 1 & 0 & 0 & 0 & 0 & 1 & 0 & 8 & \\
25 & 1 & 1 & 0 & 1 & 1 & 1 & 1 & 8 & \\
26 & 1 & 

## Z2 x Z2

In [84]:
## mu 0 1 2 3, eta 10 20 21 30 31 32
H22 = [[1 0 0 0 0 0 0 0 0 0], [0 1 0 0 1 0 0 0 0 0], [0 1 0 0 0 0 0 0 0 1], [0 1 0 0 -1 0 0 0 0 0], [0 1 0 0 0 0 0 0 0 -1], [0 0 1 0 0 1 0 0 0 0], [0 0 1 0 0 0 0 0 1 0], [0 0 1 0 0 -1 0 0 0 0], [0 0 1 0 0 0 0 0 -1 0], [0 0 0 1 0 0 1 0 0 0], [0 0 0 1 0 0 0 1 0 0], [0 0 0 1 0 0 -1 0 0 0], [0 0 0 1 0 0 0 -1 0 0]]

13-element Vector{Matrix{Int64}}:
 [1 0 … 0 0]
 [0 1 … 0 0]
 [0 1 … 0 1]
 [0 1 … 0 0]
 [0 1 … 0 -1]
 [0 0 … 0 0]
 [0 0 … 1 0]
 [0 0 … 0 0]
 [0 0 … -1 0]
 [0 0 … 0 0]
 [0 0 … 0 0]
 [0 0 … 0 0]
 [0 0 … 0 0]

In [ ]:
groupStructure = 4
@time s22 = sample([2, 2], H22, 2^20); ## get the sample points (< 2 seconds usually)
println(length(s22))
ranks = [getRank(s, n) for s in collect(values(s22))]; ## get the rank for each sample point
R = Set(ranks)
println("R=", R)
counts = [sum([r==k for r in ranks]) for k in R]; ## tabulate the ranks
priors = [c/sum(counts) for c in counts]; ## compute priors (% of samples with given rank)
counts